In [3]:
# Train model
from modelpretrained import NN
from customDataModulepretrained import CustomDataModule
import configpretrained
from callbacks import MyPrintingCallback, EarlyStopping
import pytorch_lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import seed_everything
import torch
import torchvision.models as models

seed_everything(
    42, workers=True
)  # By setting workers=True in seed_everything(), Lightning derives unique seeds across all dataloader workers and processes for torch, numpy and stdlib random number generators. When turned on, it ensures that e.g. data augmentations are not repeated across workers.



if __name__ == "__main__":
    
    model = models.vgg16(weights='VGG16_Weights.DEFAULT')
    
    logger = TensorBoardLogger("tb_logs", name="my_model") # tb_logs is the folder, name is the name of the experiment/model
    model = NN(
        model=model,
        input_shape=configpretrained.INPUT_SHAPE,
        num_classes=configpretrained.NUM_CLASSES,
        learning_rate=configpretrained.LEARNING_RATE,
    )  # .to(device)
    dm = CustomDataModulepretrained(
        data_dir=configpretrained.DATA_DIR,
        train_csv=configpretrained.TRAIN_CSV_1,
        val_csv=configpretrained.VAL_CSV_1,
        test_csv=configpretrained.TEST_CSV,
        batch_size=configpretrained.BATCH_SIZE,
        num_workers=configpretrained.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        logger=logger, # PyTorch lightning will automatically know what we are logging by looking at our model.py logs
        accelerator=configpretrained.ACCELERATOR,
        devices=configpretrained.DEVICES,
        min_epochs=configpretrained.MIN_EPOCHS,
        max_epochs=configpretrained.MAX_EPOCHS,
        deterministic=configpretrained.DETERMINISTIC#,
#        callbacks=[MyPrintingCallback(), EarlyStopping(monitor="val_loss")],
    )  # deterministic ensures random seed reproducibility

    trainer.fit(model, dm)  # it will automatically know which dataloader to use

# A general place to start is to set num_workers equal to the number of CPU cores on that machine. You can get the number of CPU cores in python using os.cpu_count(), but note that depending on your batch size, you may overflow RAM memory.


Global seed set to 42


TypeError: Value after * must be an iterable, not int